In [62]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Sample DataFrames
df1 = pd.DataFrame({'ref_num': ['12 FAM 041.1', '12 FAM 041.2', '12 FAM 043.1', '12 FAM 043.2-1\xa0', '12 FAM 081 (a)', '5 FAH-7 H-142', '5 FAM 741']})
df2 = pd.DataFrame({'ref_num': ['12 FAM 041.1', '12 FAM 041.2', '12 FAM 043.1', '12 FAM 043.2-1 (a)', '12 FAM 081 (a)', '5 FAH-7 H-142', '5 FAM 741']})

In [ ]:
# df1 will represent the dirty values, and may have some cleaning functions applied to them

df1['ref_num'] = df1['ref_num'].str.replace(r'[^A-Zb-z0-9]', '', regex=True)
df1['ref_num'] = df1['ref_num'].str.replace('.', '')
df1['ref_num'] = df1['ref_num'].str.replace('5 FAH-7 H-142', 'H-142')

,ref_num
0,12FAM0411
1,12FAM0412
2,12FAM0431
3,12FAM04321
4,12FAM081
5,5FAH7H142
6,5FAM741


In [ ]:
def fuzzy_lookup(source_df, target_df, source_col, target_col) -> pd.DataFrame():
    '''
        source_df: dataframe with the column/strings to lookup
        target_df: dataframe where you want to find matching values
        returns: dataframe
    '''
    threshold = .7
    matches = []
    for source_val in source_df[source_col]:
        best_match = process.extractOne(
            source_val,
            target_df[target_col],
            scorer=fuzz.ratio
        )
        if best_match and best_match[1] >= threshold:
            matches.append((source_val, best_match[0], best_match[1]))
        else:
            matches.append((source_val, None, None))
    return pd.DataFrame(matches, columns=[source_col, f'{target_col}_match', 'similarity'])

fuzzy_lookup(df1, df2, 'ref_num', 'ref_num')



,ref_num,ref_num_match,similarity
0,12FAM0411,12 FAM 041.1,86
1,12FAM0412,12 FAM 041.2,86
2,12FAM0431,12 FAM 043.1,86
3,12FAM04321,12 FAM 043.1,82
4,12FAM081,12 FAM 081 (a),76
5,5FAH7H142,5 FAH-7 H-142,82
6,5FAM741,5 FAM 741,88
